<a href="https://colab.research.google.com/github/niansia/Stocker/blob/master/%E3%80%8C%E3%80%8C3_Data_preprocessing_ipynb%E3%80%8D%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 01
import pandas as pd
import numpy as np

data = {
    'size': ['M','S',np.nan,'M','XL'],
    'color': ['green', 'blue', 'blue', np.nan, np.nan],
    'price': [200, np.nan, 200, 300, 300],
    'quantity': [np.nan, 35000, np.nan, 20000, 10000]
}
X = pd.DataFrame(data)
X_orig = X.copy()
X.style.highlight_null(null_color='yellow')

In [ ]:
# 02
X.info()

In [ ]:
# 03
X_col_num = ['price','quantity']
X_num = X[X_col_num]
X_num.style.highlight_null(null_color='yellow')

In [ ]:
# 04
from sklearn.impute import SimpleImputer
si = SimpleImputer(strategy='mean')
X_num_impute = si.fit_transform(X_num)
X_num_impute

In [ ]:
# 05
si.statistics_

In [ ]:
# 06
X_num.fillna(X_num.mean())

In [ ]:
# 07
from sklearn.preprocessing import StandardScaler, MinMaxScaler
ss = StandardScaler()
X_num_impute_ss = ss.fit_transform(X_num_impute)
X_num_impute_ss

In [ ]:
# 08
df_X_num_impute = pd.DataFrame(X_num_impute)
(df_X_num_impute - df_X_num_impute.mean())/df_X_num_impute.std(ddof=0)

In [ ]:
# 09
from sklearn.pipeline import make_pipeline
num_pl = make_pipeline(SimpleImputer(strategy='mean'),
                       StandardScaler())
num_pl.fit_transform(X_num)

In [ ]:
# 10
X_col_cat = ['size','color']
X_cat = X[X_col_cat]
X_cat.style.highlight_null(color='yellow')

In [ ]:
# 11
si = SimpleImputer(strategy='most_frequent')
X_cat_impute = si.fit_transform(X_cat)
X_cat_impute

In [ ]:
X_cat

In [ ]:
# 12
pd.get_dummies(X_cat)

In [ ]:
X_cat_impute

In [ ]:
# 13
from sklearn.preprocessing import OneHotEncoder
oh = OneHotEncoder(sparse=False)
X_cat_impute_oh = oh.fit_transform(X_cat_impute)
X_cat_impute_oh

In [ ]:
# 14
oh.get_feature_names_out(['size','color'])

In [ ]:
# 15
pd.DataFrame(X_cat_impute_oh, columns=oh.get_feature_names_out(['size','color']))

In [ ]:
cat_pl=make_pipeline(SimpleImputer(strategy='most_frequent'),OneHotEncoder(sparse=False))
cat_pl.fit_transform(X_cat)


In [ ]:
# 17
oh_in_pl = cat_pl.named_steps['onehotencoder']
oh_in_pl.get_feature_names_out(['size','color'])

In [ ]:
# 18
np.concatenate([X_num_impute_ss, X_cat_impute_oh], axis=1).round(2)

In [ ]:
# 19
from sklearn.compose import ColumnTransformer
data_pl = ColumnTransformer([
    ('num_pl', num_pl, X_col_num),
    ('cat_pl', cat_pl, X_col_cat)
])
data_pl.fit_transform(X).round(2)

In [ ]:
# 20
from sklearn.compose import ColumnTransformer
data_pl = ColumnTransformer([
    ('num_pl', 'drop', X_col_num),
    ('cat_pl', cat_pl, X_col_cat)
])
pd.DataFrame(data_pl.fit_transform(X))

In [ ]:
# 21
data_pl = ColumnTransformer([
    ('num_pl', num_pl, ['price']),
    ('cat_pl', cat_pl, X_col_cat)
], remainder='passthrough')
pd.DataFrame(data_pl.fit_transform(X))

In [ ]:
# 22
data_pl = ColumnTransformer([
    ('num_pl', num_pl, ['price']),
    ('cat_pl', cat_pl, X_col_cat)
], remainder='drop')
pd.DataFrame(data_pl.fit_transform(X))

In [ ]:
# 23
from sklearn.compose import ColumnTransformer
data_pl = ColumnTransformer([
    ('num_pl', SimpleImputer(strategy='mean'), X_col_num),
    ('cat_pl', cat_pl, X_col_cat)
])
pd.DataFrame(data_pl.fit_transform(X))

In [ ]:
# 24
# 第一步：取得cat_pl管道器
data_pl.named_transformers_['cat_pl']

In [ ]:
# 25
# 第二步：取得onehotencoder欄位對應結果
data_pl.named_transformers_['cat_pl'].\
named_steps['onehotencoder'].get_feature_names_out()

In [ ]:
# 26
# 你如何知道管道器裡的轉換器名稱呢？make_pipeline會自動小寫轉換器的名稱當索引鍵。
# 如果還是不確定就用named_steps.keys()列出所有的索引鍵值
data_pl.named_transformers_['cat_pl'].named_steps.keys()

In [ ]:
# 27
# 第三步：將所有欄位整理到DataFrame裡
X_col_cat_oh = data_pl.named_transformers_['cat_pl'].\
named_steps['onehotencoder'].get_feature_names_out(X_col_cat)
columns = X_col_num + X_col_cat_oh.tolist()
print('整合後的欄位資料：',columns)
pd.DataFrame(data_pl.fit_transform(X), columns=columns)

In [ ]:
# 28
# 資料
df_full = pd.DataFrame({'price':[10,20,30,40,10,20]})
print('原始資料\n', df_full)

from sklearn.preprocessing import KBinsDiscretizer
kb = KBinsDiscretizer(n_bins=3, encode='ordinal')
kb.fit_transform(df_full)

In [ ]:
# 29
si = SimpleImputer(strategy='constant', fill_value='Missing')
X_cat_impute = si.fit_transform(X_cat)
X_cat_impute

In [ ]:
# 30
num_pl = make_pipeline(SimpleImputer(strategy='mean'), StandardScaler())
num_pl.set_params(standardscaler=None)
num_pl.fit_transform(X_num)

In [ ]:
# 31
# 檢視管道器裡的結果也沒有出現standardscaler了
num_pl.named_steps

In [ ]:
# 32
from sklearn.preprocessing import MinMaxScaler
num_pl = make_pipeline(SimpleImputer(strategy='mean'), StandardScaler())
num_pl.set_params(standardscaler=MinMaxScaler())
num_pl.fit_transform(X_num)